In [69]:
# 필요 라이브러리 불러오기
import os
import logging
import numpy as np
import pandas as pd
import polars as pl
import itertools
from tqdm.notebook import tqdm

import dask.bag as db
import dask.dataframe as dd
from dask import delayed

from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# Configure logging
logging.basicConfig(level=logging.INFO)

## Configuration and Credentials

In [128]:
# ================================
# Configuration and Credentials
# ================================

# Retrieve credentials from environment variables or use defaults
GLOBUS_USER = os.getenv("GLOBUS_USER", "won103203@naver.com")
GLOBUS_PW = os.getenv("GLOBUS_PW", "goTeamC@1")

# Path to Microsoft Edge WebDriver
edge_driver_path = "C:/Users/yuhan/Downloads/edgedriver_win64/msedgedriver.exe"

# Edge options
edge_options = Options()
# Uncomment the following line to run in headless mode (recommended for automation)
# edge_options.add_argument('--headless')
edge_options.add_argument('--no-sandbox')
edge_options.add_argument('--disable-dev-shm-usage')

## Selenium Helper Functions

In [138]:
def create_edge_driver():
    """Creates and returns a new Edge WebDriver instance with the specified options."""
    service = Service(edge_driver_path)
    driver = webdriver.Edge(service=service, options=edge_options)
    driver.implicitly_wait(5)  # Set an implicit wait
    return driver

def login_to_globus(driver, user=GLOBUS_USER, password=GLOBUS_PW):
    """
    Logs in to the Glo-Bus website using the provided driver and credentials.
    Adjust the element selectors to match the actual site's login form.
    """
    try:
        driver.get("https://www.glo-bus.com/")
        # Wait for the login button or form to load (update the selector as needed)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "loginbutton"))
        )
        logging.info("Login page loaded.")
        
        # Selectors for the username and password fields
        username_field = driver.find_element(By.ID, "acct_name")
        password_field = driver.find_element(By.ID, "passwdInput")
        username_field.clear()
        username_field.send_keys(user)
        password_field.clear()
        password_field.send_keys(password)
        
        # Click the login submit button
        driver.find_element(By.ID, 'loginbutton').click()
        
        # Wait for a post-login element to confirm success (update selector)
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, "text-nowrap"))
        )
        logging.info("Login successful.")
    except Exception as e:
        logging.error("Error during login: %s", e)
        raise

def go_to_decision_page(driver, subsection):
    """
    Navigates to the decision page after logging in.
    Adjust the element selectors to match the actual site's structure.
    """
    try:
        # Click on the decision page link (update the selector)
        # driver.get(f'https://www.glo-bus.com/users/program21/decisions/{subsection}')
        driver.find_element(By.LINK_TEXT, "Go to Decisions/Reports").click()
        
        # Wait for the decision page to open in a new tab (update the expected number of windows)
        # WebDriverWait(driver, 3).until(
        #     EC.number_of_windows_to_be(2)
        # )

        # Switch to most recently opened tab
        driver.switch_to.window(driver.window_handles[-1])        

        # Wait for the decision page to load
        WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, "/html/body/glo-app/section/div/div[1]/glo-main-header/nav/div/div[1]/breadcrumb/nav/ol/li/h1"))
        )

        logging.info(f"Decision page loaded for {subsection}.")
    except Exception as e:
        logging.error("Error navigating to decision page: %s", e)
        raise


In [ ]:
# Define Product Design Options for Action-Capture Camera and UAV Drone
action_capture_camera_design_dict = {
    ('Image Sensor Size', 'G4-btn'): ['8mm', '9mm', '10mm', '11mm', '12mm', '13mm', '14mm'],
    ('LCD Display Screen', 'G5-btn'): ['230k', '460k', '610k', '920k', '1040k', '1230k', '2360k'],
    ('Image Quality', 'G6-btn'): ['1920×1080', '1920×1440', '2704×1520', '2704×2028', '3840×2160', '3840×2400', '4096×2160'],
    ('Number of Photo Modes', 'G7-btn'): ['4 / 3', '6 / 3', '7 / 3', '8 / 3', '10 / 4', '12 / 4', '16 / 4'],
    ('Camera Housing', 'G9-btn'): ['4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'],
    ('Editing / Sharing Capabilities', 'G10-btn'): ['4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16'],
    ('Included Accessories', 'G11-btn'): ['6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20'],
    ('Extra Performance Features', 'G13-btn'): ['2', '3', '4', '5', '6', '7', '8', '9', '10'],
    ('Number of Action Capture Camera Models', 'G14-btn'): ['1', '2', '3', '4', '5', '6', '7'],
    ('Product R&D Expenditures', 'G16-btn'): ['0', '3000', '6000', '9000', '12000', '16000', '20000', '24000', '28000', '32000', '36000', '40000', '45000', '50000']
}

uav_drone_design_dict = {
    ('Built-In Camera', 'G22-btn'): ['No Upgrade', 'Minor Upgrade', 'Significant Upgrade', 'Major Upgrade'],
    ('GPS / WiFi / Bluetooth', 'G23-btn'): ['Basic', 'Enhanced', 'Advanced', 'Best Available'],
    ('Battery Pack', 'G24-btn'): ['8', '10', '12', '15', '18', '21', '25', '30'],
    ('Number of Rotors', 'G25-btn'): ['4', '6', '8'],
    ('Rotor Performance / Flight Controller', 'G26-btn'): ['Basic', 'Enhanced', 'Advanced', 'Best Available'],
    ('Body Frame Construction', 'G27-btn'): ['High-Strength Plastic', 'g10/FR4 Fiberglass', 'Carbon Fiber'],
    ('Obstacle Sensors', 'G28-btn'): ['Front Only', 'Front/Rear', '360° Basic', '360° Enhanced', '360° Advanced', '360° Best Available'],
    ('Camera Stabilization Device', 'G29-btn'): ['Basic', 'Enhanced', 'Advanced', 'Best Available'],
    ('Extra Performance Features', 'G31-btn'): ['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'],
    ('Number of UAV Drone Models', 'G32-btn'): ['1', '2', '3', '4', '5', '6', '7'],
    ('Product R&D Expenditures', 'G34-btn'): ['0', '3000', '6000', '9000', '12000', '15000', '18000', '21000', '24000', '27000', '30000', '35000', '40000', '45000', '50000']
}

def calculate_number_of_design_options(design_dict):
    """
    Returns the number of design options for a given design dictionary.
    """

    # Calculate the number of design options
    num_design_options = 1
    for key in design_dict.keys():
        num_design_options *= len(design_dict[key])

    return num_design_options

# Calculate the number of design options for each product
print(f"Number of Action-Capture Camera Design Options: {calculate_number_of_design_options(action_capture_camera_design_dict)}")
print(f"Number of UAV Drone Design Options: {calculate_number_of_design_options(uav_drone_design_dict)}")

Number of Action-Capture Camera Design Options: 5368323870
Number of UAV Drone Design Options: 162570240


In [82]:
def create_random_combinations(design_dict, num_combinations=1000):
    """
    Generates random combinations of design options based on the provided design dictionary.
    The number of combinations is determined by the specified multiple of 1000.
    
    Parameters:
        design_dict (dict): Dictionary of design options.
        num_comb_thousands (int): Number of combinations in thousands (default is 100, i.e. 100,000 combos).
    
    Returns:
        pd.DataFrame: DataFrame containing randomly generated combinations.
    """

    # Create a dictionary where keys are the option names and the values are random choices
    data = {}
    for (option, btn) in design_dict:
        choices = design_dict[(option, btn)]
        data[btn] = np.random.choice(choices, size=num_combinations)
    
    return pd.DataFrame(data)

### Action Capture Camera
- 하위 3개 조건 다 제거 후 돌리기
- Camera Housing: > 13
- Editing/Sharing Capabilities > 13
- Included Accessories > 13

- 단 모델 수는 4개 이하로 제한



### UAV Drone minimum settings

- GPS / Wifi / Basic: Basic unincluded
- Rotor Performance/ Flight Controller: Basic unincluded
- Battery Pack: 15
- Number of Rotors: 8
- R&D Expenditures: > 24000

In [ ]:
def fix_design_choices(design_dict, fix_values_dict):
    """
    design_dict: 디자인 선택지 딕셔너리
    fix_values_dict: 고정할 선택지 딕셔너리
    design_dict에서 fix_values_dict의 key에 해당하는 선택지를 values로 고정
    새로운 선택지 딕셔너리를 반환
    """
    new_design_dict = design_dict.copy()
    for key, value in fix_values_dict.items():
        new_design_dict[key] = [value]
    return new_design_dict

In [115]:
strategy_A_camera_year_6_dict ={
    ('Image Sensor Size', 'G4-btn'): ['11mm', '12mm', '13mm', '14mm'],
    ('LCD Display Screen', 'G5-btn'): ['920k', '1040k', '1230k', '2360k'],
    ('Image Quality', 'G6-btn'): ['2704×2028', '3840×2160', '3840×2400', '4096×2160'],
    ('Number of Photo Modes', 'G7-btn'): ['8 / 3', '10 / 4', '12 / 4', '16 / 4'],
    ('Camera Housing', 'G9-btn'): ['13', '14', '15', '16'],
    ('Editing / Sharing Capabilities', 'G10-btn'): ['13', '14', '15', '16'],
    ('Included Accessories', 'G11-btn'): ['13', '14', '15', '16', '17', '18', '19', '20'],
    ('Extra Performance Features', 'G13-btn'): ['5', '6', '7', '8', '9', '10'],
    ('Number of Action Capture Camera Models', 'G14-btn'): ['1', '2', '3', '4'],
    ('Product R&D Expenditures', 'G16-btn'): ['20000', '24000', '28000', '32000', '36000', '40000', '45000', '50000']
}

strategy_A_drone_year_6_dict = {
    ('Built-In Camera', 'G22-btn'): ['No Upgrade', 'Minor Upgrade', 'Significant Upgrade', 'Major Upgrade'],
    ('GPS / WiFi / Bluetooth', 'G23-btn'): ['Enhanced', 'Advanced', 'Best Available'],
    ('Battery Pack', 'G24-btn'): ['15', '18', '21', '25', '30'],
    ('Number of Rotors', 'G25-btn'): ['8'],
    ('Rotor Performance / Flight Controller', 'G26-btn'): ['Enhanced', 'Advanced', 'Best Available'],
    ('Body Frame Construction', 'G27-btn'): ['High-Strength Plastic', 'g10/FR4 Fiberglass', 'Carbon Fiber'],
    ('Obstacle Sensors', 'G28-btn'): ['Front Only', 'Front/Rear', '360° Basic', '360° Enhanced', '360° Advanced', '360° Best Available'],
    ('Camera Stabilization Device', 'G29-btn'): ['Enhanced', 'Advanced', 'Best Available'],
    ('Extra Performance Features', 'G31-btn'): ['5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'],
    ('Number of UAV Drone Models', 'G32-btn'): ['1', '2', '3', '4', '5', '6', '7'],
    ('Product R&D Expenditures', 'G34-btn'): ['35000', '40000', '45000', '50000']
}

# Calculate number of design options after implementing strategy A
print(f"Number of Action-Capture Camera Design Options (Strategy A): {calculate_number_of_design_options(strategy_A_camera_year_6_dict)}")
print(f"Number of UAV Drone Design Options (Strategy A): {calculate_number_of_design_options(strategy_A_drone_year_6_dict)}")

Number of Action-Capture Camera Design Options (Strategy A): 6291456
Number of UAV Drone Design Options (Strategy A): 2993760


## Random Options Sampling

### for UAV Drone

In [147]:
strat_A_rand_drone_options_df = create_random_combinations(strategy_A_drone_year_6_dict, num_combinations=2000)
strat_A_rand_drone_options_df.head()

,G22-btn,G23-btn,G24-btn,G25-btn,G26-btn,G27-btn,G28-btn,G29-btn,G31-btn,G32-btn,G34-btn
0,Major Upgrade,Advanced,15,8,Best Available,g10/FR4 Fiberglass,360° Advanced,Best Available,12,1,40000
1,No Upgrade,Advanced,15,8,Enhanced,High-Strength Plastic,Front Only,Advanced,12,2,50000
2,Minor Upgrade,Enhanced,21,8,Advanced,g10/FR4 Fiberglass,360° Best Available,Advanced,5,4,45000
3,Major Upgrade,Advanced,25,8,Enhanced,Carbon Fiber,Front Only,Advanced,5,2,50000
4,Minor Upgrade,Advanced,25,8,Enhanced,Carbon Fiber,360° Basic,Enhanced,13,4,35000


In [ ]:
# Example usage for small set of options
strat_A_rand_camera_options_df_results = scrape_pq_and_cost_mini_parallel_threads(strat_A_rand_drone_options_df, 'drone', chunk_size=200)

INFO:root:Login page loaded.
INFO:root:Login successful.
INFO:root:Decision page loaded for product-design.
INFO:root:Decision page loaded for product-design.


Simulating Design Parameters:   0%|          | 0/200 [00:00<?, ?it/s]

INFO:root:Decision page loaded for product-design.


Simulating Design Parameters:   0%|          | 0/200 [00:00<?, ?it/s]

INFO:root:Decision page loaded for product-design.
INFO:root:Decision page loaded for product-design.


...

INFO:root:Decision page loaded for product-design.
INFO:root:Decision page loaded for product-design.


Simulating Design Parameters:   0%|          | 0/200 [00:00<?, ?it/s]

Simulating Design Parameters:   0%|          | 0/200 [00:00<?, ?it/s]

Simulating Design Parameters:   0%|          | 0/200 [00:00<?, ?it/s]

ERROR:root:Error during scraping: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
ERROR:root:Error during scraping: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))
ERROR:root:Error during scraping: HTTPConnectionPool(host='localhost', port=52494): Max retries exceeded with url: /session/9bbdd668f8aac452c9b8817868f8b9a1/element/f.21F3AC4F357C28CCBD204B7503DDD83B.d.7BE002088918942DEA16369CD288D711.e.51/enabled (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001890A62E850>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
ERROR:root:Error during scraping: HTTPConnectionPool(host='localhost', port=52494): Max retries exceeded with url: /session/9bbdd668f8aac452c9b8817868f8b9a1/element/f.21F3AC4F357C2

In [146]:
strat_A_rand_camera_options_df_results

,G22-btn,G23-btn,G24-btn,G25-btn,G26-btn,G27-btn,G28-btn,G29-btn,G31-btn,G32-btn,G34-btn,Performance / Quality Rating,Component/Feature Cost per Unit,Production Cost per Unit,Net Profit
5,Significant Upgrade,Best Available,15,8,Advanced,Carbon Fiber,360° Best Available,Best Available,14,1,40000,6.2,874.47,"1,420.02","23,304"
6,Significant Upgrade,Enhanced,21,8,Enhanced,g10/FR4 Fiberglass,360° Advanced,Advanced,8,6,50000,5.1,919.82,"1,447.71","20,743"
7,No Upgrade,Best Available,15,8,Enhanced,Carbon Fiber,Front/Rear,Enhanced,10,7,45000,3.1,905.48,"1,390.98","29,648"
8,Significant Upgrade,Enhanced,30,8,Enhanced,g10/FR4 Fiberglass,360° Advanced,Advanced,9,6,40000,4.8,"1,036.48","1,540.01","11,568"
9,No Upgrade,Enhanced,18,8,Advanced,Carbon Fiber,360° Basic,Advanced,6,2,50000,4.9,840.81,"1,364.23","28,700"
10,Minor Upgrade,Best Available,15,8,Advanced,Carbon Fiber,360° Enhanced,Advanced,8,1,45000,6.2,874.47,"1,420.02","23,304"
11,Minor Upgrade,Advanced,15,8,Best Available,Carbon Fiber,360° Enhanced,Advanced,12,1,45000,5.1,919.82,"1,447.71","20,743"
12,Minor Upgrade,Advanced,30,8,Enhanced,Carbon Fiber,Front/Rear,Enhanced,11,7,40000,6.4,905.48,"1,354.25","29,648"
13,Major Upgrade,Best Available,15,8,Best Available,Carbon Fiber,Front/Rear,Best Available,8,1,45000,6.8,"1,036.48","1,507.77","14,818"
14,Major Upgrade,Advanced,18,8,Best Available,Carbon Fiber,360° Enhanced,Best Available,15,6,45000,5.5,840.81,"1,340.71","30,668"


In [123]:
# Calculate statistics of interest
def calculate_result_ratios(results_df):
    """
    Calculates the Performance / Quality Rating Efficiency and Fixed Cost Ratio based on the provided results DataFrame.
    """
    # Change the data type of the columns to float
    # Deal with commas in the data
    result_columns = ['Performance / Quality Rating', 'Component/Feature Cost per Unit', 'Production Cost per Unit', 'Net Profit']
    results_df[result_columns] = results_df[result_columns].replace(',', '', regex=True)
    results_df[result_columns] = results_df[result_columns].astype(float)

    # Calculate the Performance / Quality Rating Efficiency and Fixed Cost Ratio
    results_df['Rating_Efficiency'] = results_df['Performance / Quality Rating'] / results_df['Production Cost per Unit']
    results_df['Fixed Cost Ratio'] = results_df['Component/Feature Cost per Unit'] / results_df['Production Cost per Unit']

    return results_df

In [124]:
strat_A_rand_camera_options_df_results = calculate_result_ratios(strat_A_rand_camera_options_df_results)

strat_A_rand_camera_options_df_results.sort_values(by='Rating_Efficiency', ascending=False).head(20)

,G22-btn,G23-btn,G24-btn,G25-btn,G26-btn,G27-btn,G28-btn,G29-btn,G31-btn,G32-btn,G34-btn,Performance / Quality Rating,Component/Feature Cost per Unit,Production Cost per Unit,Net Profit,Rating_Efficiency,Fixed Cost Ratio
1710,Significant Upgrade,Best Available,30,8,Best Available,Carbon Fiber,360° Advanced,Enhanced,8,1,35000,7.4,1097.48,1512.43,13917.0,0.004893,0.725640
1722,Major Upgrade,Advanced,21,8,Best Available,Carbon Fiber,360° Best Available,Best Available,6,1,35000,7.0,1010.46,1436.26,21853.0,0.004874,0.703536
7,Significant Upgrade,Advanced,30,8,Enhanced,Carbon Fiber,360° Enhanced,Advanced,8,1,35000,6.9,1000.47,1429.77,22502.0,0.004826,0.699742
1173,Minor Upgrade,Best Available,21,8,Advanced,Carbon Fiber,360° Basic,Best Available,12,1,35000,7.2,1075.81,1495.72,15746.0,0.004814,0.719259
1309,Major Upgrade,Enhanced,25,8,Best Available,Carbon Fiber,Front/Rear,Advanced,13,1,35000,7.4,1122.48,1537.43,11311.0,0.004813,0.730102
393,Minor Upgrade,Advanced,30,8,Best Available,g10/FR4 Fiberglass,360° Basic,Best Available,8,1,35000,6.9,1004.81,1434.11,22069.0,0.004811,0.700651
1818,Major Upgrade,Best Available,25,8,Advanced,Carbon Fiber,360° Advanced,Best Available,12,1,45000,8.1,1221.48,1685.19,-7342.0,0.004807,0.724832
1396,Major Upgrade,Advanced,25,8,Best Available,Carbon Fiber,360° Advanced,Enhanced,9,1,45000,7.5,1083.47,1561.45,8714.0,0.004803,0.693887
1099,Major Upgrade,Enhanced,30,8,Best Available,Carbon Fiber,360° Advanced,Enhanced,9,1,35000,7.3,1103.47,1520.45,13142.0,0.004801,0.725752
1604,Minor Upgrade,Enhanced,25,8,Advanced,Carbon Fiber,Front/Rear,Advanced,12,1,35000,6.9,1008.81,1438.11,21666.0,0.004798,0.701483


## Generate All Options

### Small Option Space - Less than 10 million

In [40]:
# Create a list of all possible design combinations for the action-capture camera
def generate_dataframe_from_design_dict(design_dict, mode):
    """
    Generates a DataFrame with all possible design combinations based on the given design dictionary.
    """
    if mode == 'polars':
        # Define the schema for the Polar DataFrame
        schema = {key[1]: str for key in design_dict.keys()}

        # Create a list of all possible design combinations (use generator for memory efficiency)
        design_combinations = itertools.product(*design_dict.values())

        # Create a polars DataFrame with the design combinations
        df = pl.DataFrame(design_combinations, schema=schema)

        return df
    elif mode == 'pandas':
        # Define columns for the pandas DataFrame
        columns = [key[1] for key in design_dict.keys()]

        # Create a list of all possible design combinations (use generator for memory efficiency)
        design_combinations = itertools.product(*design_dict.values())

        # Create a pandas DataFrame with the design combinations
        df = pd.DataFrame(design_combinations, columns=columns)

        return df

In [41]:
strat_A_camera_options_df = generate_dataframe_from_design_dict(strategy_A_camera_year_6_dict, mode='pandas')
strat_A_drone_options_df = generate_dataframe_from_design_dict(strategy_A_drone_year_6_dict, mode='pandas')

In [42]:
strat_A_camera_options_df

,G4-btn,G5-btn,G6-btn,G7-btn,G9-btn,G10-btn,G11-btn,G13-btn,G14-btn,G16-btn
0,11mm,920k,2704×2028,8 / 3,13,13,13,5,1,20000
1,11mm,920k,2704×2028,8 / 3,13,13,13,5,1,24000
2,11mm,920k,2704×2028,8 / 3,13,13,13,5,1,28000
3,11mm,920k,2704×2028,8 / 3,13,13,13,5,1,32000
4,11mm,920k,2704×2028,8 / 3,13,13,13,5,1,36000
...,...,...,...,...,...,...,...,...,...,...
6291451,14mm,2360k,4096×2160,16 / 4,16,16,20,10,4,32000
6291452,14mm,2360k,4096×2160,16 / 4,16,16,20,10,4,36000
6291453,14mm,2360k,4096×2160,16 / 4,16,16,20,10,4,40000
6291454,14mm,2360k,4096×2160,16 / 4,16,16,20,10,4,45000


### Large Option Space - Greater than 10 million

In [ ]:
def generate_parquet_from_design_dict(design_dict, product, output_file, npartitions=1000):
    """
    Generates a Parquet file containing all combinations of design options from the given design dictionary.

    Parameters:
    - design_dict (dict): Dictionary where keys are tuples (column_name, element_id) and values are lists of options.
    - product (str): Name of the product (used to locate the directory).
    - output_file (str): Path to the output Parquet file.
    - npartitions (int): Number of partitions for the Dask Bag.
    """
    # Define save directory
    save_dir = f"product_design_query/{product}/" + output_file

    # Extract column names (using the first element of each key) and lists of values
    cols = [key[0] for key in design_dict.keys()]
    values_lists = list(design_dict.values())

    # Create a lazy iterator for the Cartesian product of all design options
    all_combinations = itertools.product(*values_lists)
    print(all_combinations)

    # Build a Dask Bag from the iterator
    bag = db.from_sequence(all_combinations, npartitions=npartitions)
    print("Dask bag created")

    # Convert the Bag to a Dask DataFrame with the proper column names
    ddf = bag.to_dataframe(columns=cols)
    # print first 5 rows
    print(ddf.head())

    # Save the lazy Dask DataFrame as a Parquet file
    ddf.to_parquet(save_dir, write_index=False)

# Example usage
generate_parquet_from_design_dict(uav_drone_design_dict, 'drone', "all_combinations.parquet", npartitions=163)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x0000018689A988D0>>
Traceback (most recent call last):
  File "c:\miniconda\envs\WebCrawling_3.11\Lib\site-packages\ipykernel\ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [ ]:
generate_parquet_from_design_dict(action_capture_camera_design_dict, 'camera', "all_combinations.parquet", npartitions=5369)

In [ ]:
driver = create_edge_driver()
login_to_globus(driver)
go_to_decision_page(driver, 'product-design')
driver.quit()

INFO:root:Login page loaded.
INFO:root:Login successful.
INFO:root:Decision page loaded for product-design.


In [15]:
def generate_chunks(value_lists, columns, chunk_size):
    """
    Generator function that yields DataFrame chunks of the Cartesian product.
    """
    # Create an iterator for all possible combinations
    iterator = itertools.product(*value_lists)
    while True:
        # Grab the next chunk_size combinations
        chunk = list(itertools.islice(iterator, chunk_size))
        if not chunk:
            break
        yield pd.DataFrame(chunk, columns=columns)

def generate_design_combinations(product, design_dir_name, design_dict, chunk_size=1000000):
    """
    Returns a list of dictionaries, where each dictionary represents a unique design combination.
    """
    # Extract column names (first element of each key tuple) and value lists
    columns = [key[1] for key in design_dict.keys()]
    value_lists = list(design_dict.values())

    # Wrap each chunk with dask.delayed so Dask can build a lazy graph
    delayed_dfs = [delayed(chunk_df) for chunk_df in tqdm(generate_chunks(value_lists, columns, chunk_size))]

    # Create a Dask DataFrame from the delayed chunks
    ddf = dd.from_delayed(delayed_dfs)

    # Write the Dask DataFrame to a Parquet file.
    # This will write the data in partitions (chunks) to the '{product}/{design_dir_name}.parquet' directory.
    ddf.to_parquet(f'product_design_query/{product}/{design_dir_name}.parquet', write_index=False)

    print(f'Finished generating design combinations for {product} using {design_dir_name}.')

In [9]:
generate_design_combinations('drone', 'all_combinations', uav_drone_design_dict)

Finished generating design combinations for drone using all_combinations.


## Analyze Different Product Design Options

In [10]:
# read in first 5 rows of the generated parquet file
ddf = dd.read_parquet('product_design_query/drone/all_combinations.parquet')
print(ddf.head())

      G22-btn G23-btn G24-btn G25-btn G26-btn                G27-btn  \
0  No Upgrade   Basic       8       4   Basic  High-Strength Plastic   
1  No Upgrade   Basic       8       4   Basic  High-Strength Plastic   
2  No Upgrade   Basic       8       4   Basic  High-Strength Plastic   
3  No Upgrade   Basic       8       4   Basic  High-Strength Plastic   
4  No Upgrade   Basic       8       4   Basic  High-Strength Plastic   

      G28-btn G29-btn G31-btn G32-btn G34-btn  
0  Front Only   Basic       2       1       0  
1  Front Only   Basic       2       1    3000  
2  Front Only   Basic       2       1    6000  
3  Front Only   Basic       2       1    9000  
4  Front Only   Basic       2       1   12000  


In [118]:
def scrape_PD_results(driver, product):
    if product == 'action_camera':
        pq_rating_css = 'body > glo-app > section > div > div:nth-child(2) > glo-decisions > div > div.m-3 > glo-dec-product-design > div > div:nth-child(1) > div.card.mb-2.ac-calc-area1.perf-quality > div > table > tbody > tr > td.align-middle > div > span:nth-child(1) > strong'
        cf_cost_per_unit_css = 'body > glo-app > section > div > div:nth-child(2) > glo-decisions > div > div.m-3 > glo-dec-product-design > div > div:nth-child(1) > div.card.mb-2.ac-calc-area2 > div > table > tbody > tr:nth-child(9) > td:nth-child(3) > div > span:nth-child(2) > strong'
        pa_cost_per_unit_css = 'body > glo-app > section > div > div:nth-child(2) > glo-decisions > div > div.m-3 > glo-dec-product-design > div > div:nth-child(1) > div.card.mb-2.ac-calc-area2 > div > table > tbody > tr:nth-child(15) > td:nth-child(3) > div > span:nth-child(2) > strong'
        # pq_rating_xpath = '/html/body/glo-app/section/div[2]/div/glo-decisions/div/div[1]/glo-dec-product-design/div/div[1]/div[3]/div/table/tbody/tr/td[2]/div/span[1]/strong'
        # pa_cost_per_unit_xpath = '/html/body/glo-app/section/div[2]/div/glo-decisions/div/div[1]/glo-dec-product-design/div/div[1]/div[4]/div/table/tbody/tr[15]/td[3]/div/span[2]/strong'

    elif product == 'drone':
        pq_rating_css = 'body > glo-app > section > div > div:nth-child(2) > glo-decisions > div > div.m-3 > glo-dec-product-design > div > div:nth-child(2) > div.card.mb-2.uav-calc-area1.perf-quality > div > table > tbody > tr > td.align-middle > div > span:nth-child(1) > strong'
        cf_cost_per_unit_css = 'body > glo-app > section > div > div:nth-child(2) > glo-decisions > div > div.m-3 > glo-dec-product-design > div > div:nth-child(2) > div.card.mb-2.uav-calc-area2 > div > table > tbody > tr:nth-child(9) > td:nth-child(3) > div > span:nth-child(2) > strong'
        pa_cost_per_unit_css = 'body > glo-app > section > div > div:nth-child(2) > glo-decisions > div > div.m-3 > glo-dec-product-design > div > div:nth-child(2) > div.card.mb-2.uav-calc-area2 > div > table > tbody > tr:nth-child(15) > td:nth-child(3) > div > span:nth-child(2) > strong'
        # pq_rating_xpath = '/html/body/glo-app/section/div/div[2]/glo-decisions/div/div[1]/glo-dec-product-design/div/div[2]/div[3]/div/table/tbody/tr/td[2]/div/span[1]/strong'
        # pa_cost_per_unit_xpath = '/html/body/glo-app/section/div/div[2]/glo-decisions/div/div[1]/glo-dec-product-design/div/div[2]/div[4]/div/table/tbody/tr[15]/td[3]/div/span[2]/strong'
    
    net_profit_css = 'body > glo-app > section > div > div:nth-child(2) > glo-decisions > div > glo-projected-performance-top > div > div > div.col-12.d-flex.justify-content-evenly.align-items-start > table:nth-child(3) > tbody > tr:nth-child(2) > td:nth-child(2)'

    try:
        # Wait for the page to load
        # WebDriverWait(driver, 0.05).until(EC.presence_of_element_located((By.CSS_SELECTOR, pq_rating_css)))
        # WebDriverWait(driver, 0.05).until(EC.presence_of_element_located((By.CSS_SELECTOR, cf_cost_per_unit_css)))
        # WebDriverWait(driver, 0.05).until(EC.presence_of_element_located((By.CSS_SELECTOR, pa_cost_per_unit_css)))
        WebDriverWait(driver, 1).until(EC.presence_of_element_located((By.CSS_SELECTOR, net_profit_css)))

        pq_rating = driver.find_element(By.CSS_SELECTOR, pq_rating_css).text
        cf_cost_per_unit = driver.find_element(By.CSS_SELECTOR, cf_cost_per_unit_css).text
        pa_cost_per_unit = driver.find_element(By.CSS_SELECTOR, pa_cost_per_unit_css).text
        net_profit = driver.find_element(By.CSS_SELECTOR, net_profit_css).text

        # For debugging
        # print(f"Retrieved Performance Statistics for {product}: {pq_rating}, {pa_cost_per_unit}")

    except Exception as e:
        logging.error(f"Error retrieving Performance Statistics for{product}: {e}")
        pq_rating = None
        cf_cost_per_unit = None
        pa_cost_per_unit = None
        net_profit = None

    return pq_rating, cf_cost_per_unit, pa_cost_per_unit, net_profit

In [119]:
# Use selenium to change parameters in the product design page
# Each set of parameters is a row in the DataFrame
# For each row, we will:
# 1. Set the parameters
# 2. Read the PQ Rating and PA Cost per Unit
# 3. Save the results to a new column in the datafram

def set_design_parameters(chunk, driver, product):
    # List to store results for each row
    results = []

    # Iterate over each row in the chunk
    for _, row in tqdm(chunk.iterrows(), total=len(chunk), desc='Simulating Design Parameters'):
        try:
            # Set the design parameters
            for key, value in row.items():
                # Update the design parameters
                element = driver.find_element(By.ID, key)
                select = Select(element)
                select.select_by_visible_text(value)

            # Scrape the results
            pq_rating, cf_cost_per_unit, pa_cost_per_unit, net_profit = scrape_PD_results(driver, product)
            
            # Append the results as a dictionary
            results.append({
                'Performance / Quality Rating': pq_rating,
                'Component/Feature Cost per Unit': cf_cost_per_unit,
                'Production Cost per Unit': pa_cost_per_unit,
                'Net Profit': net_profit
            })
        except Exception as e:
            logging.error("Error during scraping: %s", e)
            # Append a row with NaN values in case of an error
            results.append({
                'Performance / Quality Rating': None,
                'Component/Feature Cost per Unit': None,
                'Production Cost per Unit': None,
                'Net Profit': None
            })

    # Create a DataFrame from the results and concatenate it with the original chunk
    results_df = pd.DataFrame(results, index=chunk.index)
    return pd.concat([chunk, results_df], axis=1)

# Create a function to scrape the performance quality rating and production cost per unit for each design combination
def scrape_performance_quality_and_cost(product, design_dir_name):
    """
    Scrapes the performance quality rating and production cost per unit for each design combination.
    """
    # Load the Dask DataFrame from the Parquet file
    ddf = dd.read_parquet(f'product_design_query/{product}/{design_dir_name}.parquet')

    # List to store updated delayed partitions
    updated_partitions = []

    # Iterate over each chunk of the Dask DataFrame
    for i, delayed_obj in tqdm(enumerate(ddf.to_delayed()), desc='Processing Chunks', total=ddf.npartitions):
        try:
            # Create a new Edge WebDriver instance for each chunk
            driver = create_edge_driver()

            # Login to the Glo-Bus website
            login_to_globus(driver)

            # Navigate to the product design page
            go_to_decision_page(driver, 'product-design')

            # Compute the partition to get a pandas DataFrame
            df_partition = delayed_obj.compute()

            # Replace chunk with updated version
            updated_df = set_design_parameters(df_partition, driver)

            # Wrap the updated DataFrame back into a delayed object
            updated_delayed = delayed(updated_df)
            updated_partitions.append(updated_delayed)
            
        except Exception as e:
            logging.error(f"Error processing chunk {i}: {e}")
        finally:
            # Quit the WebDriver instance
            driver.quit()
            logging.info("WebDriver instance closed.")

    # Reconstruct a Dask DataFrame from the list of delayed partitions
    updated_ddf = dd.from_delayed(updated_partitions)

    # Write the updated Dask DataFrame to a new Parquet file
    updated_ddf.to_parquet(f'product_design_query/{product}/{design_dir_name}_results.parquet', write_index=False)
    logging.info(f"Finished scraping performance quality and cost for {product} using {design_dir_name}.")

### Simulate Action Capture Camera

In [ ]:
def scrape_pq_and_cost_mini(df, product):
    try: 
        # Create a new Edge WebDriver instance for each chunk
        driver = create_edge_driver()

        # Login to the Glo-Bus website
        login_to_globus(driver)

        # Navigate to the product design page
        go_to_decision_page(driver, 'product-design')

        # Iterate over each row in the Polars DataFrame
        updated_df = set_design_parameters(df, driver, product)
    except Exception as e:
        logging.error(f"Error while scraping {product}: {e}")
    finally:
        # Quit the WebDriver instance
        driver.quit()
        logging.info("WebDriver instance closed.")

    return updated_df

## Multithread Processing

In [143]:
def process_chunk_thread(chunk_df, product, driver):
    """
    Process a chunk of the DataFrame using a thread.
    Each thread creates its own WebDriver instance.
    """
    try:
        go_to_decision_page(driver, 'product-design')
        updated_chunk = set_design_parameters(chunk_df, driver, product)
    except Exception as e:
        logging.error(f"Error processing chunk for {product}: {e}")
        updated_chunk = chunk_df
    return updated_chunk

In [ ]:
import concurrent.futures
from contextlib import contextmanager

@contextmanager
def scrape_pq_and_cost_mini_parallel_threads(df, product, chunk_size=1000):
    """
    Splits the Polars DataFrame into chunks and processes each chunk in parallel using threads.
    This avoids the pickling issues common with multiprocessing in Jupyter notebooks.
    """
    num_rows = df.shape[0]
    # Split DataFrame into chunks
    chunks = [df[i:i+chunk_size] for i in range(0, num_rows, chunk_size)]
    
    try:
        # Create a new Edge WebDriver instance for each thread
        driver = create_edge_driver()

        # Login to the Glo-Bus website
        login_to_globus(driver)

        updated_chunks = []
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(process_chunk_thread, chunk, product, driver) for chunk in chunks]
            for future in concurrent.futures.as_completed(futures):
                updated_chunks.append(future.result())
    except Exception as e:
        logging.error(f"Error while scraping {product} via Multi-thread: {e}")
    finally:
        # Quit the WebDriver instance
        driver.quit()
        logging.info("WebDriver instance closed.")
    
    updated_df = pd.concat(updated_chunks)
    return updated_df

In [ ]:
# Example usage for small set of options
strat_A_drone_options_df_results = scrape_pq_and_cost_mini(strat_A_drone_options_df, 'drone')

### Simulate UAV Drone

In [26]:
import itertools
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import gc

# Set a chunk size that suits your system's memory. Adjust as necessary.
def generate_chunks(value_lists, columns, chunk_size):
    """
    Generator function that yields DataFrame chunks of the Cartesian product.
    """
    # Create an iterator for all possible combinations
    iterator = itertools.product(*value_lists)
    while True:
        # Grab the next chunk_size combinations
        chunk = list(itertools.islice(iterator, chunk_size))
        if not chunk:
            break
        yield pd.DataFrame(chunk, columns=columns)

def generate_design_combinations_v2(product, design_dict, chunk_size=10000000):

    # Extract column names (from the first element of each key) and lists of options
    columns = [key[0] for key in design_dict.keys()]
    value_lists = list(design_dict.values())

    # Process each chunk one at a time
    for i, df_chunk in tqdm(enumerate(generate_chunks(value_lists, columns, chunk_size))):
        # Convert the chunk to a PyArrow Table
        table = pa.Table.from_pandas(df_chunk)
        # Write the table to a Parquet file (each chunk in its own file)
        pq.write_table(table, f"product_design_query/{product}/all_combinations/chunk_{i}.parquet")
        
        # Clear the DataFrame from memory and trigger garbage collection
        del df_chunk
        gc.collect()
        
        # print(f"Chunk {i} written and memory cleared.")

In [27]:
generate_design_combinations_v2('camera', action_capture_camera_design_dict)

0it [00:00, ?it/s]

In [ ]:
driver = create_edge_driver()
login_to_globus(driver)
go_to_decision_page(driver, 'product-design')
driver.quit()

In [10]:
# Read in the Parquet file as a Dask DataFrame
ddf = dd.read_parquet('product_design_query/drone/all_combinations.parquet')

# Choose only first 100 rows for testing
ddf_test = ddf.head(100)

In [ ]:
def click_resume_button():
    # Resume 버튼 클릭
    resume_button_xpath = '/html/body/glo-app/section/glo-keep-alive/div/div/div/div[3]/button'
    if driver.find_element(By.XPATH, resume_button_xpath).is_displayed():
        driver.find_element(By.XPATH, resume_button_xpath).click()
    else:
        pass

In [ ]:
click_resume_button()

In [ ]:
driver.quit()